In [4]:
from matplotlib import pyplot as plt
import pandas as pd
import glob
import json
import requests
import numpy as np
import seaborn as sns
sns.set()
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from io import StringIO
import time

In [55]:
startDate = '2020-02-01T00:00:00Z'
endDate = '2022-03-08T00:00:00Z'
var = ('TI0_0,TI0_10,TI0_20,TI0_30,TI0_40,TI0_50,TI0_60,TI0_70,TI0_80,TI0_90,TI0_100,'
       'TI0_110,TI0_120,TI0_130,TI0_140,TI0_150,TI0_170,TI0_190,TI0_210,TI0_230,'
       'TI0_250,TI0_300,TI0_350,TI0_400,TI0_450,TI0_500,TI0_600,TI0_700,TI0_800,'
       'TI0_900,TI0_1000,TI0_1100,TI0_1200,TI0_1300')
url = 'https://api.npolar.no/weather/troll/'
parameters = {
    'q': '',
    'format': 'json',
    'filter-station_id': 'kldata/nationalnr=92100/type=504',
    'filter-measured': startDate+'..'+endDate,
    'fields': 'measured,'+var,
    'limit': '14400',
    'start': '0'
}
r = requests.get(url, params=parameters)
data = pd.read_json(StringIO(r.text),orient='records')
d = pd.DataFrame(data['feed']['entries'])
d.measured = pd.to_datetime(d.measured)
d = d.set_index('measured')
#while data['feed']['list']['next']:
startDateLoop = (pd.to_datetime(endDate)-pd.to_timedelta('20D')).strftime('%Y-%m-%dT%H:%M:%SZ')
for start in pd.date_range(startDateLoop, startDate, freq='-10D'):
    end = (start+pd.to_timedelta('10D')).strftime('%Y-%m-%dT%H:%M:%SZ')
    parameters = {
        'q': '',
        'format': 'json',
        'filter-station_id': 'kldata/nationalnr=92100/type=504',
        'filter-measured': start.strftime('%Y-%m-%dT%H:%M:%SZ')+'..'+end,
        'fields': 'measured,'+var,
        'limit': '14400',
        'start': '0'
    }
    r = requests.get(url, params=parameters)
    data = pd.read_json(StringIO(r.text),orient='records')
    if data['feed']['opensearch']['totalResults']:
        d1 = pd.DataFrame(data['feed']['entries'])
        d1.measured = pd.to_datetime(d1.measured)
        d1 = d1.set_index('measured')
        d = d.append(d1,sort=False)
    
#    r = requests.get(data['feed']['list']['next'])
#    data = pd.read_json(StringIO(r.text),orient='records')
#    d1 = pd.DataFrame(data['feed']['entries'])
#    d1.measured = pd.to_datetime(d1.measured)
#    d1 = d1.set_index('measured')
#    d = d.append(d1,sort=False)
d[d==-99.9] = np.nan
d.info()
d.head()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1028462 entries, 2022-03-07 23:59:00+00:00 to 2020-02-07 00:00:00+00:00
Data columns (total 34 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   TI0_1000  805002 non-null  float64
 1   TI0_210   805001 non-null  float64
 2   TI0_110   805004 non-null  float64
 3   TI0_1300  805000 non-null  float64
 4   TI0_250   805001 non-null  float64
 5   TI0_150   805000 non-null  float64
 6   TI0_1200  804999 non-null  float64
 7   TI0_450   802886 non-null  float64
 8   TI0_230   805001 non-null  float64
 9   TI0_130   805007 non-null  float64
 10  TI0_1100  805002 non-null  float64
 11  TI0_350   805004 non-null  float64
 12  TI0_90    805019 non-null  float64
 13  TI0_190   805002 non-null  float64
 14  TI0_170   805000 non-null  float64
 15  TI0_50    805002 non-null  float64
 16  TI0_0     310132 non-null  float64
 17  TI0_70    805015 non-null  float64
 18  TI0_20    805014 non-null  float64
 1

,TI0_1000,TI0_210,TI0_110,TI0_1300,TI0_250,TI0_150,TI0_1200,TI0_450,TI0_230,TI0_130,...,TI0_60,TI0_80,TI0_10,TI0_30,TI0_900,TI0_800,TI0_500,TI0_400,TI0_700,TI0_600
measured,,,,,,,,,,,,,,,,,,,,,
2022-03-07 23:59:00+00:00,-22.91,-17.50,-17.38,-22.72,-17.88,-17.31,-22.78,-19.98,-17.69,-17.31,...,-17.5,-17.44,-17.06,-17.38,-22.78,-22.4,-20.49,-19.47,-22.15,-21.45
2022-03-07 23:58:00+00:00,-22.91,-17.57,-17.38,-22.72,-17.88,-17.25,-22.78,-19.98,-17.69,-17.25,...,-17.5,-17.44,-16.99,-17.38,-22.78,-22.4,-20.49,-19.47,-22.15,-21.38
2022-03-07 23:57:00+00:00,-22.97,-17.50,-17.38,-22.72,-17.88,-17.31,-22.78,-19.98,-17.69,-17.25,...,-17.5,-17.44,-16.99,-17.38,-22.78,-22.4,-20.49,-19.47,-22.08,-21.45
2022-03-07 23:56:00+00:00,-22.97,-17.50,-17.38,-22.72,-17.88,-17.31,-22.78,-19.98,-17.69,-17.25,...,-17.5,-17.44,-16.99,-17.38,-22.78,-22.4,-20.49,-19.47,-22.08,-21.45
2022-03-07 23:55:00+00:00,-22.91,-17.50,-17.38,-22.72,-17.88,-17.31,-22.78,-19.98,-17.69,-17.25,...,-17.5,-17.44,-17.06,-17.38,-22.78,-22.4,-20.49,-19.47,-22.15,-21.38


In [58]:
d.tail()

,TI0_1000,TI0_210,TI0_110,TI0_1300,TI0_250,TI0_150,TI0_1200,TI0_450,TI0_230,TI0_130,...,TI0_60,TI0_80,TI0_10,TI0_30,TI0_900,TI0_800,TI0_500,TI0_400,TI0_700,TI0_600
measured,,,,,,,,,,,,,,,,,,,,,
2020-02-07 00:04:00+00:00,-23.42,-12.36,-8.99,-23.04,-13.69,-10.26,-23.10,-19.03,-12.99,-9.57,...,-7.28,-7.98,-8.36,-6.84,-23.29,-22.91,-19.92,-17.95,-22.53,-21.51
2020-02-07 00:03:00+00:00,-23.42,-12.29,-8.99,-23.04,-13.69,-10.33,-23.10,-19.03,-12.99,-9.57,...,-7.28,-7.98,-8.36,-6.84,-23.29,-22.91,-19.92,-17.95,-22.53,-21.45
2020-02-07 00:02:00+00:00,-23.42,-12.36,-8.99,-23.04,-13.69,-10.26,-23.16,-19.03,-12.99,-9.63,...,-7.28,-7.98,-8.36,-6.90,-23.29,-22.91,-19.92,-17.95,-22.53,-21.45
2020-02-07 00:01:00+00:00,-23.42,-12.36,-8.99,-22.97,-13.69,-10.33,-23.10,-19.03,-13.06,-9.63,...,-7.28,-7.98,-8.36,-6.84,-23.29,-22.91,-19.92,-17.95,-22.53,-21.51
2020-02-07 00:00:00+00:00,-23.42,-12.36,-8.99,-22.97,-13.69,-10.33,-23.16,-19.03,-12.99,-9.57,...,-7.28,-7.98,-8.36,-6.84,-23.29,-22.91,-19.92,-17.95,-22.53,-21.45


In [13]:
taf = d.sort_index()
taf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1123200 entries, 2022-02-25 22:16:00+00:00 to 2022-03-07 23:59:00+00:00
Data columns (total 34 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   TI0_1000  1123200 non-null  float64
 1   TI0_210   1123200 non-null  float64
 2   TI0_110   1123200 non-null  float64
 3   TI0_1300  1123200 non-null  float64
 4   TI0_250   1123200 non-null  float64
 5   TI0_150   1123200 non-null  float64
 6   TI0_1200  1123200 non-null  float64
 7   TI0_450   1066338 non-null  float64
 8   TI0_230   1123200 non-null  float64
 9   TI0_130   1123200 non-null  float64
 10  TI0_1100  1123200 non-null  float64
 11  TI0_350   1123200 non-null  float64
 12  TI0_90    1123200 non-null  float64
 13  TI0_190   1123200 non-null  float64
 14  TI0_170   1123200 non-null  float64
 15  TI0_50    1123200 non-null  float64
 16  TI0_0     890370 non-null   float64
 17  TI0_70    1123200 non-null  float64
 18  TI0_20    1123200

In [50]:
start_date = pd.to_datetime('2020-2-1 0:0:0')
end_date = pd.to_datetime('2020-4-1 0:0:0')
for single_date in pd.date_range(end_date, start_date, freq='-10D'):
    print(single_date.strftime("%Y-%m-%d 00:00:00"), single_date+pd.to_timedelta('10D'))

2020-04-01 00:00:00 2020-04-11 00:00:00
2020-03-22 00:00:00 2020-04-01 00:00:00
2020-03-12 00:00:00 2020-03-22 00:00:00
2020-03-02 00:00:00 2020-03-12 00:00:00
2020-02-21 00:00:00 2020-03-02 00:00:00
2020-02-11 00:00:00 2020-02-21 00:00:00
2020-02-01 00:00:00 2020-02-11 00:00:00
